In [ ]:
### SETTINGS ####
DATA_FILE_PATH = 'Burn-Up.csv'

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
import datetime
register_matplotlib_converters()
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['lines.linewidth'] = 1.5
darkgrey = '#3A3A3A'
lightgrey = '#414141'
barblue = plt.rcParams['axes.prop_cycle'].by_key()['color'][0]
plt.rcParams['text.color'] = darkgrey
plt.rcParams['axes.labelcolor'] = darkgrey
plt.rcParams['xtick.color'] = lightgrey
plt.rcParams['ytick.color'] = lightgrey

# Comparing Burn Ups: Story Points vs Story Count

## Context

The team wants to determine if assiging story points (estimate) for work items helps drive conversations and decisions for prioritization.

## Idea

The idea is to compare the burn-up of stories by estimate (story points) to story count. Does the burn up of story estimate provide different/unique insights than simply story count?

In [ ]:
def datesWithoutTime(item):
    item['Closed Date'] = datetime.datetime.strptime(item['Closed Date'].strftime('%Y-%m-%d'), '%Y-%m-%d').date()
    return item

kanban_data = pd.read_csv(
    DATA_FILE_PATH, usecols=['Closed Date', 'Work Item Type', 'Story Points'], parse_dates=['Closed Date']
).dropna().transform(datesWithoutTime, 'columns')
kanban_data.head(1)

## Analysis

Calculate the cumulative sum of story points and story count by date of completion. This is done for only stories with an estimate.

In [ ]:
kanban_data = kanban_data.where(lambda x: x['Story Points'] > 0)
date_range = pd.date_range(
    start=kanban_data['Closed Date'].min(), end=kanban_data['Closed Date'].max())
pts_throughput = pd.crosstab(
    kanban_data['Closed Date'], kanban_data['Work Item Type'], values=kanban_data['Story Points'], aggfunc="sum", colnames=[None]).reset_index()
pts_throughput["Throughput"] = pts_throughput['User Story'].cumsum()
count_throughput = pd.crosstab(
    kanban_data['Closed Date'], kanban_data['Work Item Type'], colnames=[None]).reset_index()
count_throughput['Throughput'] = count_throughput['User Story'].cumsum()
ptsPlot = pts_throughput.plot(
    x='Closed Date', y='Throughput', linewidth=2.5, figsize=(14, 3), legend=None)
ptsPlot.set_title("Burn Up by Story Points", loc='left', fontdict={
             'fontsize': 18, 'fontweight': 'semibold'})
ptsPlot.set_xlabel('Time')
ptsPlot.set_ylabel('Story Points Completed')
ptsPlot.axhline(y=0, color=lightgrey, alpha=.5);
countPlot = count_throughput.plot(
    x='Closed Date', y='Throughput', linewidth=2.5, figsize=(14, 3), legend=None)
countPlot.set_title("Burn up by Story Count", loc='left', fontdict={
             'fontsize': 18, 'fontweight': 'semibold'})
countPlot.set_xlabel('Time')
countPlot.set_ylabel('Stories Completed')
countPlot.axhline(y=0, color=lightgrey, alpha=.5)

## Conclusion

The two burn-up charts look almost identical; bewteen the relationship of time and completed stories. For this reason, there is little evidence to suggest that there are additional insights that story points provide over story count.